In [29]:
import networkx as nx
from pyvis.network import Network
import pandas as pd
import math
import matplotlib.pyplot as plt

In [30]:
df = pd.read_excel('raw_data/cartel_shareholders/ABB.xlsx')

# drop last 3 rows
df = df[:-3]

df_cleaned = df.dropna(how='all').dropna(axis=1, how='all')

# Get relevant columns with % O/S in the name
relevant_columns_percent = df_cleaned.columns[df_cleaned.columns.str.contains('% O/S')]
relevant_columns_filing = df_cleaned.columns[df_cleaned.columns.str.contains('Filing Type')]

df_cleaned = df_cleaned[['Investor Name', 'Investor Sub-Type'] + list(relevant_columns_percent) + list(relevant_columns_filing)]

# Change column names to be more readable by renaming % O/S to perc_os_{month_year} and Filing Type to filing_type_{month_year}
col_list = []

for col in df_cleaned.columns:
    if '% O/S' in col:
        month_year = col.split(' ')[-1]
        col_list.append(f'perc_os_{month_year}')
    elif 'Filing Type' in col:
        month_year = col.split(' ')[-1]
        col_list.append(f'filing_type_{month_year}')
    else:
        col_list.append(str.lower(str.replace(col, ' ', '_')))

df_cleaned.columns = col_list

df_cleaned.head(10)

,investor_name,investor_sub-type,perc_os_31-Dec-2011,perc_os_31-Dec-2010,perc_os_31-Dec-2009,perc_os_31-Dec-2008,perc_os_31-Dec-2007,perc_os_31-Dec-2006,perc_os_31-Dec-2005,perc_os_31-Dec-2004,...,filing_type_31-Dec-2009,filing_type_31-Dec-2008,filing_type_31-Dec-2007,filing_type_31-Dec-2006,filing_type_31-Dec-2005,filing_type_31-Dec-2004,filing_type_31-Dec-2003,filing_type_31-Dec-2002,filing_type_31-Dec-2001,filing_type_31-Dec-2000
0,Ålandsbanken Asset Management Ab,Investment Advisor,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,...,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Wilmington Trust Investment Management LLC,Bank and Trust,0,0.00,0.00,NaN,NaN,NaN,NaN,NaN,...,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"William Blair & Company, L.L.C.",Investment Advisor/Hedge Fund,NaN,NaN,NaN,NaN,0.00,0.57,NaN,NaN,...,NaN,NaN,13F,13F,NaN,NaN,NaN,NaN,NaN,NaN
3,Wells Capital Management Inc.,Investment Advisor/Hedge Fund,0,0.07,0.07,NaN,NaN,NaN,NaN,NaN,...,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Warburg Invest Kapitalanlagegesellschaft mbH,Investment Advisor,NaN,NaN,0.00,0.00,NaN,NaN,NaN,NaN,...,Aggregate MFs,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Vontobel Asset Management, Inc.",Investment Advisor/Hedge Fund,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,...,NaN,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Veritas Investment Trust GmbH,Investment Advisor,NaN,NaN,NaN,0.17,0.17,NaN,NaN,NaN,...,NaN,Aggregate MFs,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"Vanguard Group, Inc.",Investment Advisor,0.01,0.00,0.25,0.20,0.17,NaN,NaN,NaN,...,Aggregate MFs,Aggregate MFs,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Universal-Investment-Gesellschaft mbH,Investment Advisor/Hedge Fund,NaN,NaN,NaN,0.00,0.01,NaN,NaN,NaN,...,NaN,Aggregate MFs,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Udwadia (D E),Individual Investor,NaN,0.00,0.00,NaN,NaN,NaN,NaN,NaN,...,Other Substantial/Declarable,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# Get all Files in raw_data/cartel_shareholders folder
import os
files = os.listdir("raw_data/cartel_shareholders")
files = [f for f in files if f.endswith(".xlsx")]

print(len(files))

# Create a list of all the dataframes
dfs = []
for f in files:
    df = pd.read_excel("raw_data/cartel_shareholders/" + f)
    
        # drop last 3 rows
    df = df[:-3]

    # Drop only empty rows
    df_cleaned = df.dropna(how='all', axis=0)

    # Get relevant columns with % O/S in the name
    relevant_columns_percent = df_cleaned.columns[df_cleaned.columns.str.contains('% O/S')]
    relevant_columns_filing = df_cleaned.columns[df_cleaned.columns.str.contains('Filing Type')]

    df_cleaned = df_cleaned[['Investor Name', 'Investor Sub-Type'] + list(relevant_columns_percent) + list(relevant_columns_filing)]

    # Change column names to be more readable by renaming % O/S to perc_os_{month_year} and Filing Type to filing_type_{month_year}
    col_list = []

    for col in df_cleaned.columns:
        if '% O/S' in col:
            month_year = col.split(' ')[-1]
            col_list.append(f'perc_os_{month_year}')
        elif 'Filing Type' in col:
            month_year = col.split(' ')[-1]
            col_list.append(f'filing_type_{month_year}')
        else:
            col_list.append(str.lower(str.replace(col, ' ', '_')))

    df_cleaned.columns = col_list

    df_cleaned["cartel_name"] = f.split(".")[0]
    print(f.split(".")[0])
    dfs.append(df_cleaned)

dfs[10].head(10)

61
Aalberts
ABB
Akzo Nobel
Alstom SA
Areva
Asahi
AU Optronics
BAM
Bayern
Boliden
British Airways
Cathay Pacific
Chemtura
Chimei
chiquita
Chungwa
Commerzbank
del monte
Dow
Elpida
ENI
EON
Exxon Mobil
Fuji Electric
Fujifilm
GDF suez
Hannstar Display
Henkel
hitachi ltd
Hitachi Maxell
ICI
IMI PLC
Infineon
LG Display
Micron
Mitsubishi
Mueller Industries
Nanya Tech
NEC
Nippon electric glass
Panasonic
Pilkington
procter gamble
Qantas
rautaruukki
Repsol YPF
samsung
SAS AB
Siemens
Singapore Airlines
SKW Stahl
Sony
Toshiba
Total
Unilever NV
Unilever PLC
Unipetrol
United technologies corp
Uralita
Whirlpool
Zeon


,investor_name,investor_sub-type,perc_os_31-Dec-2011,perc_os_31-Dec-2010,perc_os_31-Dec-2009,perc_os_31-Dec-2008,perc_os_31-Dec-2007,perc_os_31-Dec-2006,perc_os_31-Dec-2005,perc_os_31-Dec-2004,...,filing_type_31-Dec-2005,filing_type_31-Dec-2004,filing_type_31-Dec-2003,filing_type_31-Dec-2002,filing_type_31-Dec-2001,filing_type_31-Dec-2000,filing_type_31-Dec-1999,filing_type_31-Dec-1998,filing_type_31-Dec-1997,cartel_name
0,Étoile Gestion,Investment Advisor,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,...,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,NaN,Aggregate MFs,Aggregate MFs,Aggregate MFs,NaN,British Airways
1,eQ Asset Management Oy,Investment Advisor,NaN,NaN,0.00,0.00,0.00,0.00,0.00,NaN,...,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,British Airways
2,bfw fondsleitung AG,Investment Advisor,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,British Airways
3,Zürcher Kantonalbank (Asset Management),Bank and Trust,NaN,0.02,0.06,0.07,0.02,0.06,0.07,0.04,...,UK Registers,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,British Airways
4,Zurich Life Assurance plc,Insurance Company,NaN,0.09,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,British Airways
5,Wright Investors' Service Inc.,Investment Advisor,NaN,NaN,0.00,0.01,0.02,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,British Airways
6,"World Asset Management, Inc.",Investment Advisor,NaN,0.00,0.00,0.00,0.00,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,British Airways
7,Winterthur Investment Management Corporation,Insurance Company,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,British Airways
8,Wilmington Trust Investment Management LLC,Bank and Trust,NaN,NaN,NaN,NaN,NaN,NaN,0.02,0.02,...,UK Registers,UK Registers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,British Airways
9,Williams de Broë Private Investment Management...,Investment Advisor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.02,...,NaN,UK Registers,UK Registers,UK Registers,NaN,NaN,NaN,NaN,NaN,British Airways


In [32]:
print(f"Number of dataframes: {len(dfs)}")

# Merge all dataframes into one
df_merged = pd.concat(dfs)

investor_names = df_merged['investor_name'].unique()
cartel_names = df_merged['cartel_name'].unique()

# print amount of uniques
print(f"Number of unique investors: {len(investor_names)}")
print(f"Number of unique cartels: {len(cartel_names)}")

# order investors by number of companies they are in invested in
investor_counts = df_merged['investor_name'].value_counts()

investor_counts = investor_counts.sort_values(ascending=False)

print(investor_counts)

# show dataframe records where investor is Universal-Investment-Gesellschaft mbH 
df_universal = df_merged[df_merged['investor_name'] == 'Universal-Investment-Gesellschaft mbH']

df_universal.head(10)



Number of dataframes: 61
Number of unique investors: 8246
Number of unique cartels: 61
investor_name
Universal-Investment-Gesellschaft mbH       58
Norges Bank Investment Management (NBIM)    56
Vanguard Group, Inc.                        56
Dimensional Fund Advisors, LP               56
TIAA-CREF                                   56
                                            ..
C I Kasei Company Ltd.                       1
Asahi Kasei Corp                             1
Asahi Kasei Chemicals Corporation            1
First Union National Bank                    1
Euclid Advisors LLC                          1
Name: count, Length: 8246, dtype: int64


,investor_name,investor_sub-type,perc_os_31-Dec-2011,perc_os_31-Dec-2010,perc_os_31-Dec-2009,perc_os_31-Dec-2008,perc_os_31-Dec-2007,perc_os_31-Dec-2006,perc_os_31-Dec-2005,perc_os_31-Dec-2004,...,filing_type_31-Dec-2005,filing_type_31-Dec-2004,filing_type_31-Dec-2003,filing_type_31-Dec-2002,filing_type_31-Dec-2001,filing_type_31-Dec-2000,filing_type_31-Dec-1999,filing_type_31-Dec-1998,filing_type_31-Dec-1997,cartel_name
8,Universal-Investment-Gesellschaft mbH,Investment Advisor/Hedge Fund,0.01,NaN,0.00,0.34,0.29,0.57,0.63,0.15,...,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,NaN,Aggregate MFs,Aggregate MFs,NaN,NaN,Aalberts
8,Universal-Investment-Gesellschaft mbH,Investment Advisor/Hedge Fund,NaN,NaN,NaN,0.00,0.01,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ABB
71,Universal-Investment-Gesellschaft mbH,Investment Advisor/Hedge Fund,0,0.08,0.06,0.22,0.15,0.15,0.15,0.21,...,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,Akzo Nobel
45,Universal-Investment-Gesellschaft mbH,Investment Advisor/Hedge Fund,0,0.15,0.11,0.16,0.13,0.11,0.38,4.98,...,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,NaN,Alstom SA
22,Universal-Investment-Gesellschaft mbH,Investment Advisor/Hedge Fund,0,0.01,0.00,0.01,0.01,0.03,0.02,0.00,...,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,NaN,NaN,NaN,Asahi
15,Universal-Investment-Gesellschaft mbH,Investment Advisor/Hedge Fund,NaN,NaN,0.00,0.01,0.04,0.04,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AU Optronics
14,Universal-Investment-Gesellschaft mbH,Investment Advisor/Hedge Fund,0.0,0.01,0.05,0.35,0.51,0.24,0.02,0.12,...,Aggregate MFs,Aggregate MFs,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,NaN,BAM
70,Universal-Investment-Gesellschaft mbH,Investment Advisor/Hedge Fund,0.04,0.20,0.20,0.16,0.14,0.13,0.18,0.07,...,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,Aggregate MFs,Bayern
30,Universal-Investment-Gesellschaft mbH,Investment Advisor/Hedge Fund,0,0.01,0.03,0.10,0.06,0.20,0.13,0.28,...,Aggregate MFs,Aggregate MFs,NaN,NaN,NaN,NaN,NaN,Aggregate MFs,Aggregate MFs,Boliden
41,Universal-Investment-Gesellschaft mbH,Investment Advisor/Hedge Fund,NaN,0.07,0.06,0.14,0.11,0.16,0.08,0.07,...,UK Registers,UK Registers,UK Registers,UK Registers,UK Registers,UK Registers,Aggregate MFs,Aggregate MFs,Aggregate MFs,British Airways
